In [ ]:
import os
import json
import pandas as pd

def filter_reviews_incremental(parquet_file, output_file="filtered_reviews.jsonl"):
    """
    Filtra las reviews que estén en el conjunto de gmap_id provenientes del archivo parquet de restaurantes filtrados.
    Escribe los resultados línea por línea en el archivo de salida sin guardarlas todas en memoria.

    Args:
        parquet_file (str): Ruta del archivo parquet que contiene los restaurantes filtrados.
        output_file (str): Nombre del archivo de salida en formato JSON lines.
    """

    # Leer el parquet con los restaurantes filtrados
    if not os.path.exists(parquet_file):
        print(f"No se encontró el archivo {parquet_file}.")
        return

    df = pd.read_parquet(parquet_file)
    # Extraer el conjunto de gmap_ids
    gmap_ids_set = set(df["gmap_id"].dropna().unique())

    # Abrir el archivo de salida en modo escritura
    # El formato será JSON lines: cada línea un JSON con la review filtrada.
    with open(output_file, "w", encoding="utf-8") as out_f:

        # Recorrer las carpetas que empiecen con "review-"
        for dir_name in os.listdir("."):
            if os.path.isdir(dir_name) and dir_name.startswith("review-"):
                i = 1
                while True:
                    review_file = os.path.join(dir_name, f"{i}.json")
                    if not os.path.exists(review_file):
                        break
                    with open(review_file, "r", encoding="utf-8") as rf:
                        for line in rf:
                            line = line.strip()
                            if not line:
                                continue
                            try:
                                review = json.loads(line)
                            except json.JSONDecodeError:
                                continue

                            if "gmap_id" in review and review["gmap_id"] in gmap_ids_set:
                                # Escribir la línea directamente al archivo, sin almacenar en memoria
                                # Convertimos review a string nuevamente (ya viene de line, podemos usar la original)
                                # pero para asegurarnos, la reconstruimos desde el dict:
                                out_f.write(json.dumps(review, ensure_ascii=False) + "\n")
                    i += 1

    print(f"Proceso completado. Las reviews filtradas se han guardado en formato JSON lines en {output_file}.")



parquet_path = "../Data_cleaned/metadatosgoogle/filtered_top_10_states.parquet"  # Ajustar según corresponda
filter_reviews_incremental(parquet_path, "../Data_cleaned/Googl_Reviews/filtered_reviews.jsonl")
